In [21]:
import numpy as np
import gudhi as gd
from scipy.cluster.hierarchy import linkage, fcluster
import matplotlib.pyplot as plt

# Generate synthetic data (two concentric circles)
n_points = 100
angles = np.linspace(0, 2 * np.pi, n_points)
circle1 = np.column_stack([np.cos(angles), np.sin(angles)])
circle2 = 0.5 * circle1
data = np.vstack([circle1, circle2])

# Compute persistence homology with Gudhi
rips_complex = gd.RipsComplex(points=data, max_edge_length=2)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
dgms = simplex_tree.persistence()

# Extract 1-dimensional persistence diagram (loops)
loops_dgm = np.array([p[1] for p in dgms if p[0] == 1])

# Compute pairwise bottleneck distances
n_barcodes = len(loops_dgm)
distance_matrix = np.zeros((n_barcodes, n_barcodes))
for i in range(n_barcodes):
    for j in range(i+1, n_barcodes):
        distance = gd.bottleneck_distance(loops_dgm[i], loops_dgm[j])
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

# Perform hierarchical clustering with bottleneck distance
Z = linkage(distance_matrix, method='ward')
clusters = fcluster(Z, 2, criterion='maxclust')  # Define the number of clusters

# Assign each data point to its corresponding barcode's cluster
point_to_barcode_idx = np.repeat(np.arange(n_points), 2)
point_clusters = clusters[point_to_barcode_idx]

# Plot the resulting clustering
plt.figure()
plt.title('Clustering Result')
plt.scatter(data[:, 0], data[:, 1], c=point_clusters, cmap='viridis')
plt.show()


RuntimeError: Diagram must be an array of size n x 2